# Домашнее задание «Сверточные нейронные сети: практическое применение»
Преподаватель: Мария Шеянова, Даниил Корбут, Наталья Баданина, Александр Миленькин, Анастасия Успенская

    Классификация изображений: Cats vs Dogs
    Обучить модель классификации изображение на 2 класса. 
    Исходные данные и валидация решения на kaggle в рамках контеста Cats vs Dogs.
    Шаблон ipython-ноутбука для решения можно скачать по ссылке. 
    Решения необходимо прислать в виде ipython-ноутбука с указанием значения метрики на Leaderboard. 
    Задание засчитывается при значениях метрики Log Loss меньше 0.3.



In [ ]:
%%bash
pip install timm
pip install -U git+https://github.com/albumentations-team/albumentations

In [ ]:
import numpy as np 
import pandas as pd 
import os
import sys
import torch
import torch.nn as nn
import cv2
import matplotlib.pyplot as plt
import torchvision
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from torchvision import transforms
import copy
from tqdm.notebook import tqdm
import random
from PIL import Image
from pathlib import Path
import multiprocessing
from sklearn.model_selection import KFold
from zipfile import ZipFile

import timm

import albumentations as A
from albumentations.pytorch import ToTensorV2
from catalyst.contrib.nn.schedulers.onecycle import OneCycleLRWithWarmup

%matplotlib inline

In [ ]:
ZipFile('../input/dogs-vs-cats-redux-kernels-edition/train.zip',"r").extractall()
ZipFile('../input/dogs-vs-cats-redux-kernels-edition/test.zip', "r").extractall()

train_dir = Path('./train')
test_dir = Path('./test')

train_files = np.array(os.listdir(train_dir))
test_files = np.array(os.listdir(test_dir))
len(train_files), len(test_files)

In [ ]:
submit = pd.read_csv('../input/dogs-vs-cats-redux-kernels-edition/sample_submission.csv')
submit['file'] = submit.id.astype(str) + '.jpg'
submit.tail(3)

In [ ]:
train_table = pd.DataFrame(train_files, columns=['file'])
train_table['raw'] = train_table.file.str.split('.')
train_table['id'] = train_table.raw.str[1].astype(np.uint32)
train_table['label'] = (train_table.raw.str[0] == 'dog').astype(np.uint8)
train_table.label.hist()

Классы распределены равномерно!

In [ ]:
train_table = train_table.drop(columns=['raw'])
train_table

In [ ]:
NUM_CORES = multiprocessing.cpu_count()
SEED = 2021
N_SPLITS = 5
batch_size = 32
img_size = 224 #384
num_classes = 1
NUM_CORES

# Helper Functions

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

def get_img(path):
    im_bgr = cv2.imread(str(path))
    im_rgb = im_bgr[:, :, ::-1]
    return im_rgb

def get_table(files):
    table = pd.DataFrame(files, columns=['file'])
    table['raw'] = table.file.str.split('.')
    table['id'] = table.raw.str[1].astype(np.uint32)
    table['label'] = (table.raw.str[0] == 'dog').astype(np.uint8)
    table = table.drop(columns=['raw'])
    return table

def show_batch(loader):
    images, labels = next(iter(loader))
    examples = images['image'].shape[0]
    width = int(examples ** .5)
    height = examples // width
    _indexes = [(i, j) for i in range(height) for j in range(width)]
    f, ax = plt.subplots(height, width, figsize=(22, 22))
    for (i, j), img, label in zip(_indexes, images['image'], labels):
        ax[i, j].imshow(img.permute(1, 2, 0).numpy().astype(np.uint8))
        if label == 0:
            label = 'cat'
        else:
            label = 'dog'
        ax[i, j].set_title(label)
    f.tight_layout()

def inverse_normalize(tensor,
                      mean=torch.tensor([0.485, 0.456, 0.406]),
                      std=torch.tensor([0.229, 0.224, 0.225])):
    for t, m, s in zip(tensor, mean, std):
        t.mul_(s).add_(m)
    return tensor

threshold = 0.5

def show_batch(loader, predict=False, threshold=0.5):
    images, labels = next(iter(loader))
    examples = images['image'].shape[0]
    width = int(examples ** .5)
    height = examples // width
    _indexes = [(i, j) for i in range(height) for j in range(width)]
    f, ax = plt.subplots(height, width, figsize=(22, 22))

    if predict:
        preds = model(images['image'].to(device))
        if num_classes != 1:
            labels_pred = preds.argmax(dim=1)
        else:
            labels_pred = torch.sigmoid(preds) >= threshold

    else:
        labels_pred = ['unpredicted' for i in range(examples)]

    for (i, j), img, label, pred in zip(_indexes, images['image'], labels, labels_pred):
        img = inverse_normalize(img).numpy().transpose(1, 2, 0)
        ax[i, j].imshow(img)

        if label == 0:
            label = 'cat'
        if label == 1:
            label = 'dog'

        if pred == 0:
            pred = 'cat'
        if pred == 1:
            pred = 'dog'
            
        ax[i, j].set_title(label + '/' + pred)
    f.tight_layout()
    
seed_everything(SEED)

# Dataloader

In [ ]:
class CatsDataset(Dataset):
    def __init__(self, annotations, img_dir, transform=None, learning_stage='train'):
        self.img_table = annotations
        self.img_dir = img_dir
        self.transform = transform
        self.learning_stage = learning_stage

    def __len__(self):
        return self.img_table.shape[0]

    def __getitem__(self, idx):
        file_name = self.img_table.file[idx]
        img_path = os.path.join(self.img_dir, file_name)
        image = get_img(img_path)

        if self.transform:
                image = self.transform(image=image)

        if self.learning_stage in ['train', 'val']:
            label = self.img_table.label[idx]
            sample = image, label
        else:   
            sample = image, file_name

        return sample

In [ ]:
def get_train_transforms():
    return A.Compose([
                    A.Resize(img_size, img_size, interpolation=2, p=1.),
                    A.RandomSizedCrop((img_size-24, img_size-24), img_size, img_size, p=0.5),
                    A.HorizontalFlip(p=0.5),
                    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=20, p=0.5),
                    A.RGBShift(r_shift_limit=15, g_shift_limit=15, b_shift_limit=15, p=0.5),
                    A.RandomBrightnessContrast(p=0.5),
                    A.ColorJitter(p=0.3),
                    A.CoarseDropout(max_holes=3, max_height=36, max_width=36, p=0.5),
                    A.RandomGridShuffle(grid=(2, 2), p=0.2),
                    A.Blur(blur_limit=(3, 5), p=0.1),
                    A.Normalize(p=1.0), 
                    ToTensorV2(p=1.0),
                    ], p=1.)
  
def get_valid_transforms():
    return A.Compose([
                    A.Resize(img_size, img_size, interpolation=2, p=1.),
                    A.Normalize(p=1.0),
                    ToTensorV2(p=1.0),
                    ], p=1.)

In [ ]:
kfs = KFold(N_SPLITS)
test_table = submit
for train, val in kfs.split(train_files):
    #train_table = get_table(train_files[train])
    val_table = get_table(train_files[val])

In [ ]:
train_dataset = CatsDataset(train_table, train_dir, transform=get_train_transforms(), learning_stage='train')
val_dataset = CatsDataset(val_table, train_dir, transform=get_valid_transforms(), learning_stage='val')
test_dataset = CatsDataset(test_table, test_dir, transform=get_valid_transforms(), learning_stage='test')

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

loaders = {'train': train_loader,
           'val': val_loader,
           'test': test_loader,
           }

In [ ]:
show_batch(loaders['train'])

# Model

In [ ]:
class CatNet(nn.Module):
    def __init__(self, num_classes, encoder='tf_efficientnet_b5_ns', dropout=0.01):
        super().__init__()
        self.dropout = dropout
        self.backbone = timm.create_model(encoder, pretrained=True)
        self.classifier = nn.Sequential(nn.Dropout(self.dropout),
                                        nn.Linear(self.backbone.num_classes, num_classes))
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x


def get_params(model, lr=3e-3, reduce=0.1):
    return  [
        {'params': model.backbone.parameters(), 'lr': lr * reduce},
        {'params': model.classifier.parameters(), 'lr': lr},
    ]
        
model_names = ['resnet34',
               'tf_efficientnet_b1_ns',
               'tf_efficientnet_b0_ns',
               'vit_large_patch16_384',
               'tf_efficientnet_b6_ns',
               'tf_efficientnet_l2_ns_475']

model_name = model_names[2]
model = CatNet(num_classes, model_name)
param = get_params(model, lr=1e-4)

In [ ]:
def train_model(model, loss, optimizer, scheduler, num_epochs):
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs}:', flush=True)

        for phase in ['TRAIN', 'VALID']:

            if phase == 'TRAIN':
                dataloader = loaders['train']
                model.train()  
            else:
                dataloader = loaders['val']
                model.eval() 

            running_loss = 0.
            running_acc = 0.

            for image_batch, label_batch in tqdm(dataloader):
                image_batch = image_batch['image'].to(device)
                label_batch = label_batch.to(device).float().view(-1, 1)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'TRAIN'):
                    preds = model(image_batch)
                    loss_value = loss(preds, label_batch)
                    
                    if num_classes != 1:
                        preds_class = preds.argmax(dim=1)
                    else:
                        preds_class = torch.sigmoid(preds) >= threshold

                    if phase == 'TRAIN':
                        loss_value.backward()
                        optimizer.step()

                # statistics
                running_loss += loss_value.item()
                running_acc += (preds_class == label_batch.data).float().mean()

                sys.stdout.write('\x1b[1K\r')
                sys.stdout.write(f'{phase}: Current loss: {loss_value.item():.4f}')


            epoch_loss = running_loss / len(dataloader)
            epoch_acc = running_acc / len(dataloader)
            
            scheduler.step(epoch_loss)

            print(f'  Loss={epoch_loss:.4f} Acc={epoch_acc:.4f}', flush=True)

    return model

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
if num_classes != 1:
    loss = torch.nn.CrossEntropyLoss()
else:
    loss = torch.nn.BCEWithLogitsLoss()
    
optimizer = torch.optim.AdamW(param)
#scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.2, patience=2, verbose=True)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.5)

In [ ]:
model = train_model(model, loss, optimizer, scheduler, num_epochs=1)

# Predict Test

Loss=0.0105 Acc=0.9960

In [ ]:
model.eval();

In [ ]:
show_batch(loaders['test'], predict=True)

In [ ]:
file_name_list = []
pred_list = []

with torch.no_grad():
    for image_batch, file_name in tqdm(loaders['test']):
        image_batch = image_batch['image'].to(device)
        preds = model(image_batch)
        if num_classes != 1:
            labels_pred = preds.argmax(dim=1)
            preds_class = preds.argmax(dim=1)
        else:
            labels_pred = torch.sigmoid(preds) 
            preds_class = labels_pred # >= threshold

        file_name_list += [name for name in file_name]
        pred_list += [p.item() for p in preds_class]

submission = pd.DataFrame({"id":file_name_list, "label":pred_list})
#submission['label'] = submission.label.astype(int)
submission['id'] = submission.id.str.split('.')
submission['id'] = submission['id'].str[0]
submission.to_csv('submission.csv', index=False)
submission